<a href="https://colab.research.google.com/github/zhenyueli/module-21-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load the dataset
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
charity_df= pd.read_csv(url)

# Display the first few rows of the dataframe
charity_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:

# Drop non-beneficial columns
charity_df = charity_df.drop(columns=['EIN', 'NAME'])


In [7]:
# Determine the number of unique values in each column.
charity_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
charity_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [9]:
cutoff = 75
application = charity_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application[application < cutoff].index.tolist()
# Replace in dataframe
for app in application_types_to_replace:
    charity_df['APPLICATION_TYPE'] = charity_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
charity_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
charity_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification = charity_df['CLASSIFICATION'].value_counts()[charity_df['CLASSIFICATION'].value_counts() > 1]
classification

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [22]:
cutoff2 = 20
classification = charity_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification[classification <= cutoff2].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    charity_df['CLASSIFICATION'] = charity_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
charity_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [23]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_df = pd.get_dummies(charity_df)
converted_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [77]:
# Split our preprocessed data into our features and target arrays
y = converted_df['IS_SUCCESSFUL']
X = converted_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=50)

In [78]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [79]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(65, input_dim=X_train_scaled.shape[1], activation='relu'))

# Second hidden layer
nn.add(Dense(50, activation='relu'))

# New hidden layer
nn.add(Dense(20, activation='relu'))

# Add dropout

# Output layer
nn.add(Dense(1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_59 (Dense)            (None, 65)                2990      
                                                                 
 dense_60 (Dense)            (None, 50)                3300      
                                                                 
 dense_61 (Dense)            (None, 20)                1020      
                                                                 
 dense_62 (Dense)            (None, 1)                 21        
                                                                 
Total params: 7331 (28.64 KB)
Trainable params: 7331 (28.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [80]:
nn.compile(loss='binary_crossentropy',optimizer='adam',  metrics=['accuracy'])

In [81]:
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
108/108 [==============================] - 2s 4ms/step - loss: 0.5983 - accuracy: 0.7142
Epoch 2/100
108/108 [==============================] - 0s 4ms/step - loss: 0.5468 - accuracy: 0.7399
Epoch 3/100
108/108 [==============================] - 0s 3ms/step - loss: 0.5358 - accuracy: 0.7460
Epoch 4/100
108/108 [==============================] - 0s 2ms/step - loss: 0.5317 - accuracy: 0.7495
Epoch 5/100
108/108 [==============================] - 0s 2ms/step - loss: 0.5271 - accuracy: 0.7483
Epoch 6/100
108/108 [==============================] - 0s 2ms/step - loss: 0.5273 - accuracy: 0.7469
Epoch 7/100
108/108 [==============================] - 0s 2ms/step - loss: 0.5246 - accuracy: 0.7472
Epoch 8/100
108/108 [==============================] - 0s 2ms/step - loss: 0.5214 - accuracy: 0.7445
Epoch 9/100
108/108 [==============================] - 0s 2ms/step - loss: 0.5245 - accuracy: 0.7460
Epoch 10/100
108/108 [==============================] - 0s 2ms/step - loss: 0.5206 - accura

In [84]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

965/965 - 2s - loss: 0.6311 - accuracy: 0.7172 - 2s/epoch - 2ms/step
Loss: 0.6310842633247375, Accuracy: 0.7172011733055115


In [85]:
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
